In [56]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import os

In [40]:
root_data_path='../../../../Data/imgnet_localization/'

# train solution

In [41]:
train_solution=pd.read_csv(root_data_path+'LOC_train_solution.csv')

In [42]:
train_solution.head()

,ImageId,PredictionString
0,n02017213_7894,n02017213 115 49 448 294
1,n02017213_7261,n02017213 91 42 330 432
2,n02017213_5636,n02017213 230 104 414 224
3,n02017213_6132,n02017213 46 82 464 387
4,n02017213_7659,n02017213 103 66 331 335


# val solution

In [43]:
val_solution=pd.read_csv(root_data_path+'LOC_val_solution.csv')

In [44]:
val_solution.head()

,ImageId,PredictionString
0,ILSVRC2012_val_00048981,n03995372 85 1 499 272
1,ILSVRC2012_val_00037956,n03481172 131 0 499 254
2,ILSVRC2012_val_00026161,n02108000 38 0 464 280
3,ILSVRC2012_val_00026171,n03109150 0 14 216 299
4,ILSVRC2012_val_00008726,n02119789 255 142 454 329 n02119789 44 21 322 ...


# combine two csv

In [45]:
train_solution['is_val']=0
val_solution['is_val']=1

In [46]:
data_csv=pd.concat([train_solution, val_solution], ignore_index=True)

In [47]:
data_csv.head()

,ImageId,PredictionString,is_val
0,n02017213_7894,n02017213 115 49 448 294,0
1,n02017213_7261,n02017213 91 42 330 432,0
2,n02017213_5636,n02017213 230 104 414 224,0
3,n02017213_6132,n02017213 46 82 464 387,0
4,n02017213_7659,n02017213 103 66 331 335,0


# add bbox&class

In [48]:
cls=[]
bbox=[]
for index in tqdm_notebook(data_csv.index):
    preStr=data_csv.loc[index]['PredictionString'].strip().split(' ')
    cls.append(preStr[0])
    if len(preStr)==5:
        bbox.append(' '.join(preStr[1:]))
    
    else:
        bbox_preStr=[item for item in preStr if 'n' not in item]
        bbox.append(' '.join(bbox_preStr))

In [49]:
data_csv['cls']=cls
data_csv['bbox']=bbox

In [50]:
data_csv.drop(['PredictionString'],inplace=True,axis=1)

In [51]:
data_csv.head()

,ImageId,is_val,cls,bbox
0,n02017213_7894,0,n02017213,115 49 448 294
1,n02017213_7261,0,n02017213,91 42 330 432
2,n02017213_5636,0,n02017213,230 104 414 224
3,n02017213_6132,0,n02017213,46 82 464 387
4,n02017213_7659,0,n02017213,103 66 331 335


# add path

In [52]:
train_path='ILSVRC/Data/CLS-LOC/train/'
valpath='ILSVRC/Data/CLS-LOC/val/'

In [67]:
path=[]
for index in tqdm_notebook(data_csv.index):
    image_id=data_csv.loc[index]['ImageId']
    is_val=int(data_csv.loc[index]['is_val'])
    if is_val:
        path.append(valpath+image_id+'.JPEG')
    else:
        path.append(train_path+image_id.split('_')[0]+'/'+image_id+'.JPEG')

In [68]:
data_csv['path']=path

In [69]:
data_csv.head()

,ImageId,is_val,cls,bbox,path,exist
0,n02017213_7894,0,n02017213,115 49 448 294,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,0
1,n02017213_7261,0,n02017213,91 42 330 432,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,0
2,n02017213_5636,0,n02017213,230 104 414 224,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,0
3,n02017213_6132,0,n02017213,46 82 464 387,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,0
4,n02017213_7659,0,n02017213,103 66 331 335,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,0


# delete nonexist imgs

In [70]:
valid_index=[]
for index in tqdm_notebook(data_csv.index):
    img_path=root_data_path+data_csv.loc[index]['path']
    if os.path.exists(img_path):
        valid_index.append(1)
    else:
        valid_index.append(0)

In [71]:
data_csv['exist']=valid_index

In [72]:
data_csv.head()

,ImageId,is_val,cls,bbox,path,exist
0,n02017213_7894,0,n02017213,115 49 448 294,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,1
1,n02017213_7261,0,n02017213,91 42 330 432,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,1
2,n02017213_5636,0,n02017213,230 104 414 224,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,1
3,n02017213_6132,0,n02017213,46 82 464 387,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,1
4,n02017213_7659,0,n02017213,103 66 331 335,ILSVRC/Data/CLS-LOC/train/n02017213/n02017213_...,1


In [73]:
data_csv.to_csv('../save/data.csv',index=False)